# Primes with Runs

### Problem 111

Considering 4-digit primes containing repeated digits it is clear that they cannot all be the same: 1111 is divisible by 11, 2222 is divisible by 22, and so on. But there are nine 4-digit primes containing three ones:

1117, 1151, 1171, 1181, 1511, 1811, 2111, 4111, 8111

M(n, d) represents the maximum number of repeated digits for an n-digit prime where d is the repeated digit. 

N(n, d) represents the number of such primes. 

S(n, d) represents the sum of these primes.

So M(4, 1) = 3 is the maximum number of repeated digits for a 4-digit prime where one is the repeated digit, there are N(4, 1) = 9 such primes, and the sum of these primes is S(4, 1) = 22275. It turns out that for d = 0, it is only possible to have M(4, 0) = 2 repeated digits, but there are N(4, 0) = 13 such cases.

In the same way we obtain the following results for 4-digit primes.


`Digit,d    M(4, d)        N(4, d)     S(4, d)`

      0          2              13       67061 
      1 	     3               9 	  22275
      2 	     3 	          1 	   2221
      3 	     3 	         12 	  46214
      4 	     3 	          2 	   8888
      5 	     3 	          1 	   5557
      6 	     3 	          1 	   6661
      7 	     3 	          9 	  57863
      8 	     3 	          1 	   8887
      9 	     3 	          7 	  48073 


For d = 0 to 9, the sum of all S(4, d) is 273700.

Find the sum of all S(10, d).

In [1]:
using CBD, IterTools, Combinatorics, Primes
# Combinatorics source code contains many useful functions not found in README 

_L(n,ds,d) is a low level function that generates a mask of the digit 'd' of length 'n'. It then takes the cartesian product of the digits in the array 'ds' and generates all possible numbers in which the products of 'ds' have been placed in all possible positions within the mask. Finally, it checks to see if these are primes. (Need to be careful of leading 0's.)

For example, _L(9,[3,8],1), generates a mask of 9's and then generates all possible numbers in which [3,3],[3,8],[8,3] and [8,8] are placed within the mask.

This will be a unique list.

In [322]:
function _L(n,ds,d)
    result  = []
    m       = [] 
    mask    = fill(d,n)
    indxs   = combinations(1:n, length(ds))
    pds     = product(ds,ds)

    for idxs_pds = product(indxs, pds)
        m = copy(mask)
        for (idx, pd) in reduce(zip,idxs_pds)
            setindex!(m,pd,idx)
        end
        k = parse(Int64, reduce(*, map(string,m)))
        if k >= 10^(n-1) && isprime(k)
            push!(result,k)
        end
    end
    result
end

_L (generic function with 1 method)

Here are a couple of examples. 

Note that though the primes in each result are distinct, that there are duplicates between the two lists.

In [333]:
sort(_L(9, [3,8], 1))

18-element Array{Any,1}:
 111111313
 111111331
 111113113
 111113131
 111118811
 111131131
 111133111
 111311131
 111881111
 113111311
 118181111
 118811111
 131111131
 131113111
 181111181
 311111131
 811111181
 811111811

In [334]:
sort(_L(9, [3,4], 1))

18-element Array{Any,1}:
 111111313
 111111331
 111113113
 111113131
 111114131
 111131131
 111133111
 111311131
 111314111
 113111311
 113141111
 131111131
 131113111
 131114111
 141113111
 141311111
 311111131
 431111111

L(n,d,u) generates all combinations of 'u' digits, generates a mask of the digit 'd' and then generates all possible combinations of the 'u' digits substituted over this mask. 

The key function is: combinations(0:9,u), which selects 'u' digits of the digits 0..9 and depends upon _L(n,ds,d).

As shown in the example above, though each call of _L(n,ds,d) results in distinct primes, duplicates can exist between different calls to _L(n,ds,d). These must be removed.

In [325]:
function L(n,d,u)
    result = []
    for ds in combinations(0:9,u)
        r = _L(n,ds,d)
        if !isempty(r)
            append!(result,r)
        end
    end
    #VERY IMPORTANT: REMOVE DUPLICATES
    unique(result)
end

L (generic function with 1 method)

L1(n) is a top level function that generates masks for all digits and makes all possible single digit substitutions. This is useful, because for most 'n', the maximum run is simply of length (n-1). This allows us to pick those out and then dig deeper for those that has shorter maximum runs.

In [326]:
function L1(n)
    result = []
    for d in 0:9
        r = L(n,d,1)
        push!(result,r)
        end
    result 
end

L1 (generic function with 1 method)

Finally, we are ready. Calling L1(10) shows that all but three masks 'd' have maximum runs of length 9. These are {1,3,4,5,6,7,9}. 

Those 'd' that do not have maximum runs of length 9 are {0,2,8}.

In [327]:
L1(10)

10-element Array{Any,1}:
 Any[]                                                                                                                               
 [1121111111, 1111211111, 1111111121, 1111411111, 1111111411, 1151111111, 1115111111, 1111115111, 1711111111, 1117111111, 1111111181]
 Any[]                                                                                                                               
 [3333133333, 3233333333, 3333323333, 3333332333, 3333333323, 3334333333, 3333333833]                                                
 [4444444447]                                                                                                                        
 [5555555557]                                                                                                                        
 [6666666661]                                                                                                                        
 [1777777777, 7777717777, 2777777777,

The 'd' that do not have maximum runs of length 9 are {0,2,8}.

For these, we see simply start by testing if they have maximum runs of length 8. As it turns out, they do, so all we need to do is sum all of the results together.

In [329]:
L(10,0,2)

8-element Array{Int64,1}:
 6000000001
 7000000001
 1000000007
 9000000001
 1000000009
 4000000007
 4000000009
 6000000007

In [330]:
L(10,2,2)

39-element Array{Int64,1}:
 2022222221
 2202222223
 2222202223
 2222220223
 2222220227
 2232222221
 1222222223
 2222221223
 5222222221
 2222225221
 2222262221
 2222222621
 2222282221
          ⋮
 3222222229
 2232222229
 2222242229
 2222224229
 2222262227
 2222222267
 2262222229
 2222222927
 2222222297
 7222222229
 2722222229
 2272222229

In [331]:
L(10,8,2)

32-element Array{Int64,1}:
 8888808881
 8888880881
 8880888883
 8888888383
 8888880887
 8888888087
 8888088889
 8888888089
 8888888809
 8888888989
 8888882881
 8838888881
 8885888881
          ⋮
 8488888883
 8888488883
 8888888483
 8888838887
 8888888837
 8898888883
 8888848889
 8868888887
 8688888889
 8886888889
 8888868889
 9888888887

Now sum the maximum 9-run and 8-run primes (of length 10) together and we are done.

This could be easily generalized, but that's just low-level. The problem is solved.

In [332]:
@time sum(map(sum,filter(!isempty,L1(10)))) + sum(L(10,0,2)) + sum(L(10,2,2)) + sum(L(10,8,2))

  0.168103 seconds (1.43 M allocations: 64.756 MiB, 6.63% gc time)


612407567715